# CS109 - Final Project - Market Predictors

## Exploratory Data Analysis

Using [TA-Lib : Technical Analysis Library](http://ta-lib.org/)  
Also good: [Technical analysis documentation](http://tadoc.org/)

###### Notes:
I started analyzing the data provided by my company and wasn't happy with what our systems provided for several reasons--
- We had no control over the parameters used to create the features/indicators (e.g., specifying how many days used to calculate moving averages, or specifying the time periods for the Aroon Oscillator, etc.)
- It wasn't reproducible. Every data-load for a new index or investment would require a new manual feed. Fetching the data and loading from "scratch" allows us to streamline the entire process.
- The process of retrieving the manual feeds was quite laborious. They say that the best programmers are lazy ones (who write code to accomplish things rather than doing a lot of work).

I noticed that literally all of the historical data/metrics/indicators were derived from the basics (date, open, high, low, close, volume). And the basics are offered for download via API from many sources (like Yahoo Finance (used through matplotlib (!) below)).  

I thought I was going to need to implement the calculations/formulas/coding to re-create all of the features/indicators, but after a quick google search of "technical analysis python," I discovered "TA-Lib", an open-source, multi-platform library (with a python wrapper) that has already implemented more features/indicators than we can use.

To use talib, you'll have to [Install it](http://mrjbq7.github.io/ta-lib/install.html), but it is FAST and works VERY well.

- Windows user: it's a pain to compile ta-lib if environment isn't setup correctly. Get [compiled binary](http://www.lfd.uci.edu/~gohlke/pythonlibs/) and run "pip install TA_Lib-0.4.9-cp27-none-win_amd64.whl". 

You can see the flow of my explorations below, culminating in one pandas dataframe containing five full years of daily data for symbol "UXI" (ProShares Ultra Industrials--chosen at random) saved to a csv file ("UXI.csv").

There's still LOTS to do, including:
- adding signal strategies using some of the trend data
- tweaking the parameters for all of the indicators we intend to use
- adding (there are many more available) or removing features/indicators
- standardizing/normalizing the data
- etc...

But I'm planning to focus on HW5 for the next few days, so I wanted to get this into your hands as early as possible. Also--I'm writing these notes more as a diary (before I forget what I've done and the reason for doing it) and to communicate the status to you--I don't intend for this to be in the final product (as is, anyway!).

I've provided several links in this document--well worth exploring.
See you Saturday @ 8:30pm.


In [1]:
import talib
import datetime
import numpy as np
import matplotlib.colors as colors
import matplotlib.finance as finance
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

In [2]:
#Examples playing around with talib
#from: http://mrjbq7.github.io/ta-lib/index.html

from talib import MA_Type

#upper, middle, lower = talib.BBANDS(close, matype=MA_Type.T3)

#momentum = talib.MOM(close, timeperiod=5)

# note that all ndarrays must be the same length!
inputs = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}
input_arrays = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}

from talib import abstract
sma = abstract.SMA
sma = abstract.Function('sma')

from talib.abstract import *
output = SMA(input_arrays, timeperiod=25) # calculate on close prices by default
output = SMA(input_arrays, timeperiod=25, price='open') # calculate on opens
upper, middle, lower = BBANDS(input_arrays, 20, 2, 2)
slowk, slowd = STOCH(input_arrays, 5, 3, 0, 3, 0) # uses high, low, close by default
slowk, slowd = STOCH(input_arrays, 5, 3, 0, 3, 0, prices=['high', 'low', 'open'])

### Here are all the functions (and functions by group) offered through talib

#### Many details on each technical analysis indicator: [SparkCharts](http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators)

In [3]:
#talib functions
print talib.get_functions()
print talib.get_function_groups()

['ATR', 'NATR', 'TRANGE', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'BETA', 'CORREL', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGI

In [4]:
#Example of fetching Yahoo historical data
#from: http://matplotlib.org/examples/pylab_examples/finance_work2.html

import pandas as pd
startdate = datetime.date(2008, 1, 1)
today = enddate = datetime.date(2014, 12, 31)#datetime.date.today()
ticker = 'UXI'   # '^GSPC'

fh = finance.fetch_historical_yahoo(ticker, startdate, enddate)
# a numpy record array with fields: date, open, high, low, close, volume, adj_close)

r = mlab.csv2rec(fh)
fh.close()
r.sort()

In [5]:
#Copy to pandas dataframe
df = pd.DataFrame(data = r, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close'])
df['volume'] = df['volume'].astype(float)
df.head()

,date,open,high,low,close,volume,adj_close
0,2008-01-02,76.820000,76.889999,74.180000,74.180000,2000,71.376371
1,2008-01-03,74.959999,74.959999,74.959999,74.959999,100,72.126890
2,2008-01-04,72.500000,72.500000,70.650002,70.699997,1500,68.027894
3,2008-01-07,71.410004,71.410004,68.440002,68.440002,5900,65.853316
4,2008-01-08,69.769997,69.769997,66.300003,66.300003,1800,63.794198


Note: "adj_close" is close price adjusted for dividends and splits.

In [6]:
#Create new fields for various technical indicators directly from talib

#Bolinger Bands
df['bb_upper'], df['bb_middle'], df['bb_lower'] = talib.BBANDS(df['close'].values, timeperiod=20,
                                                              nbdevup=2, nbdevdn=2, matype=0)
df['bb_pct'] = (df['close'] - df['bb_lower'])/(df['bb_upper'] - df['bb_lower'])
df['bb_bandwidth'] = ( (df['bb_upper'] - df['bb_lower']) / df['bb_middle']) * 100

#Double Exponential Moving Average
df['dema50'] = talib.DEMA(df['close'].values, timeperiod=50)
df['dema100'] = talib.DEMA(df['close'].values, timeperiod=100)
df['dema200'] = talib.DEMA(df['close'].values, timeperiod=200)

#Exponential Moving Average
df['ema50'] = talib.EMA(df['close'].values, timeperiod=50)
df['ema100'] = talib.EMA(df['close'].values, timeperiod=100)
df['ema200'] = talib.EMA(df['close'].values, timeperiod=200)

#Weighted Moving Average
df['wma50'] = talib.WMA(df['close'].values, timeperiod=50)
df['wma100'] = talib.WMA(df['close'].values, timeperiod=100)
df['wma200'] = talib.WMA(df['close'].values, timeperiod=200)

#Simple Moving Average
df['sma50'] = talib.SMA(df['close'].values, timeperiod=50)
df['sma100'] = talib.SMA(df['close'].values, timeperiod=100)
df['sma200'] = talib.SMA(df['close'].values, timeperiod=200)

#Moving Average Shell
df['ma50'] = talib.MA(df['close'].values, timeperiod=50, matype=0)
df['ma100'] = talib.MA(df['close'].values, timeperiod=100, matype=0)
df['ma200'] = talib.MA(df['close'].values, timeperiod=200, matype=0)

#Kaufman Adaptive Moving Average
df['kama'] = talib.KAMA(df['close'].values, timeperiod=30)

#Parabolic SAR
df['sar'] = talib.SAR(df['high'].values, df['low'].values, acceleration=0.02, maximum=0.2)

#Average Directional Movement Index
df['adx'] = talib.ADX(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)
df['plus_di'] = talib.PLUS_DI(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)
df['minus_di'] = talib.MINUS_DI(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

#Aroon
df['aroon_dn'], df['aroon_up'] = talib.AROON(df['high'].values, df['low'].values, timeperiod=25)

#Aroon Oscillator
df['aroon_osc'] = talib.AROONOSC(df['high'].values, df['low'].values, timeperiod=25)

#Commodity Channel Index
df['cci'] = talib.CCI(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

#Moving Average Convergence/Divergence
df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(df['close'].values,
                                                           fastperiod=12, slowperiod=26, signalperiod=9)

#Percentage Price Oscillator
df['ppo'] = talib.PPO(df['close'].values, fastperiod=12, slowperiod=26, matype=0)

#Money Flow Index
df['mfi'] = talib.MFI(df['high'].values, df['low'].values, df['close'].values, df['volume'].values, timeperiod=14)

#Rate of change ratio 100 scale
df['roc'] = talib.ROCR100(df['close'].values, timeperiod=10)

#Relative Strength Index
df['rsi'] = talib.RSI(df['close'].values, timeperiod=14)

#Ultimate Oscillator
df['ult_osc'] = talib.ULTOSC(df['high'].values, df['low'].values, df['close'].values, 
                            timeperiod1=7, timeperiod2=14, timeperiod3=28)

#Williams' %R
df['willr'] = talib.WILLR(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

#Chaikin A/D Line
df['ad'] = talib.AD(df['high'].values, df['low'].values, df['close'].values, df['volume'].values)

#Chaikin A/D Oscillator
df['ad_osc'] = talib.ADOSC(df['high'].values, df['low'].values, df['close'].values, df['volume'].values,
                          fastperiod=3, slowperiod=10)

#Average True Range
df['atr'] = talib.ATR(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

#On Balance Volume
df['obv'] = talib.OBV(df['close'].values, df['volume'].values)

#Stochastic
df['stoch_slowk'], df['stoch_slowd'] = talib.STOCH(df['high'].values, df['low'].values, df['close'].values, 
                                                   fastk_period=5,
                                                    slowk_period=3,
                                                    slowk_matype=0,
                                                    slowd_period=3,
                                                    slowd_matype=0)

#Stochastic Relative Strength Index
df['stoch_fastk'], df['stoch_fastd'] = talib.STOCHRSI(df['close'].values, timeperiod=14,
                                                      fastk_period=5, fastd_period=3, fastd_matype=0)

#1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
df['trix'] = talib.TRIX(df['close'].values, timeperiod=30)

In [7]:
#Derive fields for Support and Resistance

tperiod = 30
lotrade = talib.MIN(df['low'].values, timeperiod=tperiod)
hitrade = talib.MAX(df['high'].values, timeperiod=tperiod)
df['sr_pivotpts'] = (df['high'].values + df['low'].values + df['close'].values)/3
df['sr_res1'] = 2*df['sr_pivotpts'].values - lotrade
df['sr_sup1'] = 2*df['sr_pivotpts'].values - hitrade
df['sr_res2'] = df['sr_pivotpts'].values + (hitrade - lotrade)
df['sr_sup2'] = df['sr_pivotpts'].values - (hitrade - lotrade)
df['sr_res3'] = hitrade + 2*(df['sr_pivotpts'].values - lotrade)
df['sr_sup3'] = lotrade - 2*(hitrade - df['sr_pivotpts'].values)

In [8]:
#Derive Custom Volatility Signal

rangetol1, rangetol2 = 0.005, 0.005
dv = pd.DataFrame(data = r, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close'])
dv['range'] = dv['high'].values - dv['low'].values
dv['rangepct'] = dv['range'].values/dv['low'].values
dv['prior'] = talib.SUM(dv['close'].values, timeperiod=2) - dv['close'].values
dv['perf'] = (dv['close'].values - dv['prior'].values) / dv['prior'].values
dv['r180'] = pd.rolling_mean(dv['rangepct'].values, window=180)
#dv['vtest1'] = (dv['perf'].values>0 & (dv['rangepct'].values > (dv['r180'].values + rangetol1)))
dv['vtest1a'] = dv['perf'].values > 0
dv['vtest1b'] = (dv['rangepct'].values > (dv['r180'].values + rangetol1))
dv['vtest1'] = dv['vtest1a'] & dv['vtest1b']
#dv['vtest2'] = (dv['perf'].values<0 & (dv['rangepct'].values > (dv['r180'].values + rangetol2)))
dv['vtest2a'] = dv['perf'].values < 0
dv['vtest2b'] = (dv['rangepct'].values > (dv['r180'].values + rangetol2))
dv['vtest2'] = dv['vtest2a'] & dv['vtest2b']
dv['vc'] = dv['vtest1'] | dv['vtest2']
dv['atest1'] = (pd.rolling_sum(dv['vc'], window=3) - dv['vc']) <= 1
dv['atest2'] = pd.rolling_sum(dv['vc'], window=5) == 0.
dv['atest3'] = pd.rolling_sum(dv['vc'], window=15) <= 1.
dv['atest23'] = (dv['atest2'] | dv['atest3'])
dv['in_or_out'] = dv['atest1'] & dv['atest23']
dv['in_or_out_p'] = (pd.rolling_sum(dv['in_or_out'], window=2) - dv['in_or_out']) == 1
dv['in_or_out_e'] = dv['in_or_out'] == dv['in_or_out_p']
dv['cv_signal'] = dv['in_or_out'] & dv['in_or_out_e']

df['cv_signal'] = dv['cv_signal']

In [9]:
df.describe()
df.dtypes
df.head()

,date,open,high,low,close,volume,adj_close,bb_upper,bb_middle,bb_lower,...,stoch_fastd,trix,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2008-01-02,76.820000,76.889999,74.180000,74.180000,2000,71.376371,NaN,NaN,NaN,...,NaN,NaN,75.083333,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2008-01-03,74.959999,74.959999,74.959999,74.959999,100,72.126890,NaN,NaN,NaN,...,NaN,NaN,74.959999,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2008-01-04,72.500000,72.500000,70.650002,70.699997,1500,68.027894,NaN,NaN,NaN,...,NaN,NaN,71.283333,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2008-01-07,71.410004,71.410004,68.440002,68.440002,5900,65.853316,NaN,NaN,NaN,...,NaN,NaN,69.430003,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2008-01-08,69.769997,69.769997,66.300003,66.300003,1800,63.794198,NaN,NaN,NaN,...,NaN,NaN,67.456668,NaN,NaN,NaN,NaN,NaN,NaN,False


In [10]:
df.tail()

,date,open,high,low,close,volume,adj_close,bb_upper,bb_middle,bb_lower,...,stoch_fastd,trix,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1758,2014-12-24,120.040001,120.040001,119.099998,119.099998,500,118.838773,121.842701,114.1210,106.399299,...,99.290990,0.155675,119.413332,133.356664,118.426663,134.343333,104.483331,148.286665,103.496662,False
1759,2014-12-26,118.339996,119.389999,118.339996,119.389999,600,119.128138,122.005108,114.1680,106.330892,...,99.290990,0.155469,119.039998,132.609995,117.679994,133.969999,104.109997,147.539996,102.749993,True
1760,2014-12-29,118.750000,119.970001,118.750000,119.970001,3600,119.706867,122.483677,114.3230,106.162323,...,99.290990,0.156126,119.563334,133.656667,118.726666,134.493335,104.633333,148.586668,103.796665,True
1761,2014-12-30,118.760002,118.760002,118.160004,118.660004,13000,118.399743,122.947092,114.5855,106.223908,...,66.666667,0.157110,118.526670,131.583339,116.653338,133.456671,103.596669,146.513340,101.723337,True
1762,2014-12-31,117.790001,117.790001,117.790001,117.790001,600,117.531648,123.199618,114.7230,106.246382,...,33.333333,0.158087,117.790001,130.110001,115.180000,132.720002,102.860000,145.040002,100.249999,True


In [12]:
#Delete oldest data (used only for accurate rolling periods)
df = df[df.date>datetime.date(2010, 1, 1)]

#Save df to csv
df.to_csv("data/UXI.csv", index=False)